In [1]:
# import libraries

import sys
import os
from pathlib import Path
from datetime import datetime
import time

import pandas as pd
import numpy as np
import json
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib.font_manager as fm

plt.rcParams['figure.figsize'] = (25,15)
plt.rcParams['font.family'] = 'NanumGothic'

import folium
import geopandas as gpd
import fiona
from shapely.geometry import Point, LineString
import pyproj

import warnings
warnings.filterwarnings(action='ignore')

print(sys.version)

3.7.10 (default, Feb 26 2021, 13:06:18) [MSC v.1916 64 bit (AMD64)]


In [2]:
land_use_code = pd.read_csv('data/13.부산시남구_토지이용현황(코드).csv')
land_use = gpd.read_file('data/12.부산시남구_토지이용현황.geojson')

In [3]:
land_use_code.head(5)

,mj_info,md_info,mi_info,cd,lnd_info
0,농지,논,경지정리답,1110,물을 이용하여 벼를 재배하기 위한 토지로 수로가 용수를 공급하는 경지 정리지
1,농지,논,미경지정리답,1120,"물을 이용하여 벼를 재배하기 위한 토지로서 주로 계단 형태이며, 경사가 심한 산간지..."
2,농지,밭,"보통,특수작물",1210,"물을 대지 아니하고 식물을 재배하는 토지로 보통작물인 무, 배추, 시금치 등의 채소..."
3,농지,밭,과수원 기타,1220,"과실수를 재배하는 토지로 사과, 배, 복숭아, 포도, 감, 밀감, 밤, 호두나무 등..."
4,입지,초지,자연초지,2110,자연적으로 조성된 초지로서 벌목지는 도화상에서는 자연초지로 구분을 하고 현지조사 당...


In [4]:
# cd 데이터 타입 변환 (나중에 지도로 나타내기 위해)
land_use_code['cd'].astype(str)

0     1110
1     1120
2     1210
3     1220
4     2110
5     2120
6     2210
7     2220
8     2230
9     2310
10    2330
11    2320
12    2340
13    3110
14    3120
15    3130
16    3140
17    3210
18    3220
19    3230
20    3240
21    3310
22    3320
23    3410
24    3420
25    3430
26    3440
27    3510
28    3520
29    3530
30    3540
31    3550
32    4110
33    4120
34    4210
35    4310
36    4320
37    4410
Name: cd, dtype: object

In [5]:
# 토지이용 분류

land_use_code['mj_info'].unique()

array(['농지', '입지', '도시 및 주거지', '수계'], dtype=object)

In [6]:
# 각각의 항목에 대해 자세한 이용 목적 확인 (순서대로 농지, 입지, 도시 및 주거지, 수계)
# "주차장 설치 지역은 도로, 공터 및 주택부지 등으로 주택부지는 건물 및 토지 매입 등을 전제로 주차장 위치 선정이 가능합니다."
# 각각의 항목을 확인한 후 주차장 건설에 적합한 부지 코드만 남긴다

is_farm = (land_use_code['mj_info'] == '농지')
is_green = (land_use_code['mj_info'] == '입지')
is_city = (land_use_code['mj_info'] == '도시 및 주거지')
is_water = (land_use_code['mj_info'] == '수계')

farm = land_use_code[is_farm]
green = land_use_code[is_green]
city = land_use_code[is_city]
water = land_use_code[is_water]

In [7]:
farm # 전부 제외

,mj_info,md_info,mi_info,cd,lnd_info
0,농지,논,경지정리답,1110,물을 이용하여 벼를 재배하기 위한 토지로 수로가 용수를 공급하는 경지 정리지
1,농지,논,미경지정리답,1120,"물을 이용하여 벼를 재배하기 위한 토지로서 주로 계단 형태이며, 경사가 심한 산간지..."
2,농지,밭,"보통,특수작물",1210,"물을 대지 아니하고 식물을 재배하는 토지로 보통작물인 무, 배추, 시금치 등의 채소..."
3,농지,밭,과수원 기타,1220,"과실수를 재배하는 토지로 사과, 배, 복숭아, 포도, 감, 밀감, 밤, 호두나무 등..."


In [8]:
green # 전부 제외

,mj_info,md_info,mi_info,cd,lnd_info
4,입지,초지,자연초지,2110,자연적으로 조성된 초지로서 벌목지는 도화상에서는 자연초지로 구분을 하고 현지조사 당...
5,입지,초지,인공초지,2120,축산과 낙농을 위해 사용되는 초지 및 이를 목적으로 인위적으로 조성한 초지와 도로의...
6,입지,임목지,침엽수림,2210,전체의2/3이상이 침엽수 위주로 이루어진 산림
7,입지,임목지,활엽수림,2220,전체의2/3이상이 활엽수 위주로 이루어진 산림
8,입지,임목지,혼합수림,2230,침엽수 또는 활엽수가 혼재하는 산림
9,입지,기타,골프장,2310,골프를 위하여 조성된 녹지와 해당 부대 시설
10,입지,기타,유원지,2330,"위락, 휴양 등을 위한 놀이시설이 포함된 대규모 공원(도심지내 소규모 공원은 제외하..."
11,입지,기타,공원묘지,2320,사설 또는 공공 공원묘지
12,입지,기타,암벽 및 석산,2340,"산악지 및 해안가의 바위, 암벽 등을 포함"


In [9]:
city # 3110, 3120, 3130, 3140, 3210,3320 (일반주택지, 고층주택지, 상업/업무지, 나대지/인공녹지, 도로, 공업나지)

,mj_info,md_info,mi_info,cd,lnd_info
13,도시 및 주거지,주거지 및 상업지,일반주택지,3110,"5층 미만의 연립, 다세대 및 단독 주택지(단독 주택에 일부가 점포로 사용되는 경우..."
14,도시 및 주거지,주거지 및 상업지,고층주택지,3120,5층 이상의 공동주택으로 아파트 및 해당 부대시설(100x100이하의 면적)을 포함함
15,도시 및 주거지,주거지 및 상업지,"상업,업무지",3130,"상가, 시장, 서비스업 등 매매를 위한 건물이나 은행, 사무실등 업무용 건물이 자리..."
16,도시 및 주거지,주거지 및 상업지,나대지 및 인공녹지,3140,"주거 밀집지내 공토로 존재하는 나대지 및 도심지내 소규모 공원 및 녹지, 주거밀집지..."
17,도시 및 주거지,교통시설,도로,3210,"6m 이상의 고속화도로 및 국도, 지방도, 시.군도와 갓길, 인도포함한 용지를 말하..."
18,도시 및 주거지,교통시설,철로 및 주변지역,3220,복선 이상의 철로 및 부속용지
19,도시 및 주거지,교통시설,공항,3230,"항공기 이착륙을 위한 활주로를 포함한 격납고, 승강장, 관제시설 등이 존재하는 지역"
20,도시 및 주거지,교통시설,항만,3240,선박의정박과 물품을 하역하는 시설이 존재하는 지역으로 창고시설 등을 포함
21,도시 및 주거지,공업지,공업시설,3310,제조업에 이용되는 토지 및 대규모 공업단지(상업시설과 공업시설이 혼재되어 있는 경우...
22,도시 및 주거지,공업지,"공업나지,기타",3320,"공업지대 내에 아직 시설이 들어서지 않은 대지, 공장지대 또는 공업단지로 조성된 부..."


In [10]:
city['lnd_info'].iloc[3] #나대지 및 인공녹지

'주거 밀집지내 공토로 존재하는 나대지 및 도심지내 소규모 공원 및 녹지, 주거밀집지 및 도심지 내 소규모 공원 및 녹지, 인위적으로 조성된 초지'

In [11]:
city['lnd_info'].iloc[9] # 공업나지,기타	

'공업지대 내에 아직 시설이 들어서지 않은 대지, 공장지대 또는 공업단지로 조성된 부지 중 아직 개발되지 않은 최소단위면적 이상의 나대지'

In [12]:
water # 전부 제외

,mj_info,md_info,mi_info,cd,lnd_info
32,수계,습지,갯벌,4110,"조수에 따라 그 범위가 달라질 수 있는 지역으로서 수시로 바닷물이 찰 수 있으며, ..."
33,수계,습지,염전,4120,바닷가에서 소금 생산에 이용되는 지역
34,수계,하천,하천,4210,실폭하천 및 부속토지(일반적으로 하천에서 제방까지를 포함)
35,수계,호소,"호,소",4310,"물이 고여있는 저수지, 늪"
36,수계,호소,댐,4320,발전과 홍수 예방등을 목적으로 건설된 대단위 저수관제 시설로서 제방 상단의 폭이 6...
37,수계,기타,백사장,4410,바닷가의 모래(해변 또는 사빈)등


In [13]:
# land_use 정리
# 3110, 3120, 3130, 3140, 3210,3320
# '3130' 제외 (8/3)

isin_con = land_use['UCB'].isin(['3110', '3120', '3140', '3210', '3320'])
new_land_use = land_use.loc[isin_con].reset_index(drop=True)
new_land_use['UCB'].astype(str)
new_land_use

,UCB,geometry
0,3210,"MULTIPOLYGON (((129.10241 35.14294, 129.10241 ..."
1,3110,"MULTIPOLYGON (((129.06493 35.14285, 129.06397 ..."
2,3110,"MULTIPOLYGON (((129.06629 35.14147, 129.06493 ..."
3,3110,"MULTIPOLYGON (((129.06622 35.14380, 129.06503 ..."
4,3140,"MULTIPOLYGON (((129.06619 35.14509, 129.06359 ..."
...,...,...
309,3120,"MULTIPOLYGON (((129.11639 35.11624, 129.11641 ..."
310,3110,"MULTIPOLYGON (((129.11672 35.11854, 129.11674 ..."
311,3210,"MULTIPOLYGON (((129.11756 35.12809, 129.11762 ..."
312,3210,"MULTIPOLYGON (((129.11804 35.10785, 129.11803 ..."


In [14]:
new_land_use.to_crs(epsg='4326')
land_json = new_land_use.to_json()
loc = [35.1300,129.100]

map_land = folium.Map(location=loc, tiles='cartodbpositron', zoom_start=13)

# (일반주택지, 고층주택지, 상업/업무지, 나대지/인공녹지, 도로, 공업나지) 각각 다른 색으로 나타내기 (각각 3110, 3120, 3130, 3140, 3210,3320 )

style_function = lambda x: {'fillColor': 'magenta' if x['properties']['UCB']=='3110' else \
                            'yellow' if x['properties']['UCB']=='3120' else\
                            #'red' if x['properties']['UCB'] == '3130' else\
                            'green' if x['properties']['UCB'] == '3140' else\
                            'black' if x['properties']['UCB'] == '3210'\
                            else 'blue',
                            'color':'black',
                            'weight': 0.5,
                            'stroke': False
                           }

temp = folium.GeoJson(
    land_json,
    style_function = style_function)

temp.add_child(folium.features.GeoJsonTooltip(['UCB'], labels=False))

temp.add_to(map_land)

map_land